#### HOTELS ATTRIBUTES EXTRACTION PROCESS

This notebook was used to execute manually the scripts to scrape all of the previously scrapped hotels attributes.

From the scrapping.py file we import scrape_hotels_attributes, this function takes a list of urls and returns a dataframe with the following hotel data:

- attributes
- scores

The data was scrapped from the [Booking.com](https://www.booking.com/index.es.html?label=gen173nr-1BCAEoggI46AdIM1gEaAyIAQGYAQq4ARfIAQzYAQHoAQGIAgGoAgO4At_puqsGwAIB0gIkOTkxOTFjMmEtNTEzYS00NDQyLTgxNmMtMWJmMzMyYzVjNjJj2AIF4AIB&sid=1a3dea14733e69dc474ff74d1fad21e9&keep_landing=1&sb_price_type=total&) web, from a page like this: [boooking/hotels/newyork/hotel.com](https://www.booking.com/hotel/us/west-side-ymca.es.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaAyIAQGYAQq4ARfIAQzYAQHoAQH4AQyIAgGoAgO4Arj9uqsGwAIB0gIkMzNmMDQ2ZDMtOTkwYy00ZWVmLTgwZDYtYzhjMWUxZmFjM2Yz2AIG4AIB&sid=75f60e0ba4b489f2fe2baf48281729c2&dest_id=20088325;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;hpos=4;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1701756621;srpvid=525f2b61586300dd;type=total;ucfs=1&#hotelTmpl).

In [8]:
import os
import sys
import scrapping
import pandas as pd

usa_cities = pd.read_csv("../files/data/usa_cities.csv")
usa_states = pd.read_csv("../files/data/usa_states.csv")
usa_hotels = pd.read_csv("../files/data/booking/booking_hotels.csv", index_col=0)

In [1]:
#
import re
import time
import pandas as pd

#
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService



def scrape_hotels_attributes(urls, ids, report=True):

    hotels_attributes = pd.DataFrame(columns=[
        "hotel_id", "direction", "description", "attributes", 
        "latitude", "longitude", "scores"
    ])

    # Instanciate and configurate driver
    chrome_options = selenium.webdriver.chrome.options.Options()
    #chrome_options.add_argument('--headless') # unables GUI
    chrome_options.add_argument('--disable-infobars') # unables images loading
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')  # deshabilita carga de imágenes
    driver = webdriver.Chrome(options=chrome_options)
    
    for i in range(len(urls)):

        print(f"{i+1}/{len(urls)}")
        driver.get(urls[i])
        time.sleep(2)

        try: # Obtener dirección
            direccion_element = driver.find_element(By.CLASS_NAME, "hp_address_subtitle")
            direccion_texto = direccion_element.text
        except Exception as e:
            print(f"Error obteniendo dirección para el hotel con ID {ids[i]}")
            direccion_texto = None

        try: # Obtener descripción
            descripcion_element = driver.find_element(By.CLASS_NAME, "b3efd73f69")
            descripcion_texto = descripcion_element.text
        except Exception as e:
            print(f"Error obteniendo descripción para el hotel con ID {ids[i]}")
            descripcion_texto = None

        try: # Obtener atributos (limitado a 10)
            list_items = driver.find_elements(By.CLASS_NAME, "a8b57ad3ff")[:10]
            attributes_set = set()  # Usamos un conjunto para evitar duplicados
            for li in list_items:
                attribute = li.text
                if attribute not in attributes_set:
                    attributes_set.add(attribute)
            attributes_list = list(attributes_set)  # Convertir el conjunto a lista
        except Exception as e:
            print(f"Error obteniendo atributos para el hotel con ID {ids[i]}")
            attributes_list = None

        try: # Obtener latitud y longitud
            map_element = driver.find_element(By.ID, 'hotel_sidebar_static_map')
            lat_lng_attribute = map_element.get_attribute('data-atlas-latlng')
            lat, lng = map(float, lat_lng_attribute.split(','))
        except Exception as e:
            print(f"Error obteniendo latitud y longitud para el hotel con ID {ids[i]}")
            lat, lng = None, None

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.45);") # scroll to load page and buttons
        time.sleep(2)
        score_list = []
        #try: # Scores
        scores_div = driver.find_element(By.CSS_SELECTOR, "PropertyReviewsRegionBlock")
        scores = scores_div.find_elements(By.CLASS_NAME, 'b817090550')
        for score_element in scores:
            score = score_element.text
            score = score.replace(",", ".")
            parts = score.split("\n")
            if len(parts) == 2:
                score_list.append({f"{parts[0]}": parts[1]})
        #except Exception as e:
        #    print(f"Error {type(e).__name__} obteniendo scores para el hotel con ID {ids[i]}")
        #    score_list = None
        
        # Create dataframe to concat
        attributes = pd.DataFrame({
            "url": [urls[i]],
            "direction": [direccion_texto],
            "description": [descripcion_texto],
            "attributes": [attributes_list],
            "latitude": [lat],
            "longitude": [lng],
            "scores": [score_list]})
        hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True, sort=False)

    driver.quit()
    return hotels_attributes




California

In [25]:
california_hotels = usa_hotels[usa_hotels["state"] == "Oregon"]
california_hotels_url_list = california_hotels["reviews_url"].tolist()
california_hotels_ids_list = california_hotels["hotel_id"].tolist()
len(california_hotels_ids_list)

591

In [29]:
california_hotels_attributes_1_3 = scrape_hotels_attributes(california_hotels_url_list[:20], california_hotels_ids_list[:20])
california_hotels_attributes_1_3.to_csv("../files/data/booking/attributes/oregon_hotels_attributes_1_3.csv")
california_hotels_attributes_1_3

1/20


AttributeError: 'WebDriver' object has no attribute 'find_element_by_css_selector'

In [3]:
california_hotels_attributes_2_3 = scrapping.scrape_hotels_attributes(california_hotels_url_list[400:800], california_hotels_ids_list[400:800])
california_hotels_attributes_2_3.to_csv("../files/data/booking/attributes/california_hotels_attributes_2_3.csv")
california_hotels_attributes_2_3

1/400
Error StaleElementReferenceException obteniendo scores para el hotel con ID 16609
2/400


c:\Users\david\Documents\GitHub\PF_DS_YELP\scrapping\scrapping.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True, sort=False)


Error obteniendo atributos para el hotel con ID 17059: Message: stale element reference: stale element not found
  (Session info: chrome-headless-shell=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF658324D02+56194]
	(No symbol) [0x00007FF6582904B2]
	(No symbol) [0x00007FF6581376AA]
	(No symbol) [0x00007FF65813CFC0]
	(No symbol) [0x00007FF65813F38F]
	(No symbol) [0x00007FF65813F480]
	(No symbol) [0x00007FF65817B97E]
	(No symbol) [0x00007FF6581A5E7A]
	(No symbol) [0x00007FF658176226]
	(No symbol) [0x00007FF6581A6320]
	(No symbol) [0x00007FF6581C2786]
	(No symbol) [0x00007FF6581A5C23]
	(No symbol) [0x00007FF658174A45]
	(No symbol) [0x00007FF658175AD4]
	GetHandleVerifier [0x00007FF65869D5BB+3695675]
	GetHandleVerifier [0x00007FF6586F6197+4059159]
	GetHandleVerifier [0x00007FF6586EDF63+4025827]
	GetHandleVerifier [0x00007FF6583BF

KeyboardInterrupt: 

In [ ]:
california_hotels_attributes_3_3 = scrapping.scrape_hotels_attributes(california_hotels_url_list[1000:], california_hotels_ids_list[1000:])
california_hotels_attributes_3_3.to_csv("../files/data/booking/attributes/california_hotels_attributes_3_3.csv")
california_hotels_attributes_3_3

1/556


c:\Users\david\Documents\GitHub\PF_DS_YELP\scrapping\scrapping.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True, sort=False)


2/556
3/556
4/556
5/556
6/556
7/556
8/556
9/556
10/556
11/556
12/556
13/556
14/556
15/556
16/556
17/556
18/556
19/556
20/556
21/556
22/556
23/556
24/556
25/556
26/556
27/556
28/556
29/556
30/556
31/556
32/556
33/556
34/556
35/556
36/556
37/556
38/556
39/556
40/556
41/556
42/556
43/556
44/556
45/556
46/556
47/556
48/556
49/556
50/556
51/556
52/556
53/556
54/556
55/556
56/556
57/556
58/556
59/556
60/556
61/556
62/556
63/556
64/556
65/556
66/556
67/556
68/556
69/556
70/556
71/556
72/556
73/556
74/556
75/556
76/556
77/556
Error StaleElementReferenceException obteniendo scores para el hotel con ID 28504
78/556
79/556
80/556
81/556
82/556
83/556
84/556
85/556
86/556
87/556
88/556
89/556
90/556
91/556
92/556
93/556
94/556
95/556
96/556
97/556
98/556
99/556
100/556
101/556
102/556
103/556
104/556
105/556
106/556
107/556
108/556
109/556
110/556
111/556
112/556
113/556
114/556
115/556
116/556
117/556
118/556
119/556
120/556
121/556
122/556
123/556
124/556
125/556
126/556
127/556
128/556
129/556


,hotel_id,direction,description,attributes,latitude,longitude,scores
0,26614,"1928 18th Street Apt #E, Santa Mónica, Los Áng...",Spacious Two Bedroom in Santa Monica está en L...,"[¿Hay piscina?, WiFi gratis, ¿Hay spa?, ¿Se si...",34.019617,-118.473552,[]
1,26629,"127 Ellis Street, Union Square, San Francisco,...",Hotel Abri Union Square es un alojamiento con ...,"[Parking privado, WiFi gratis, Adaptado person...",37.785364,-122.408369,"[{'Personal': '8.4'}, {'Instalaciones y servic..."
2,26645,"242 Powell Street, Union Square, San Francisco...",El Stratford Hotel tiene una buena ubicación e...,"[Recepción 24 horas, WiFi gratis, Adaptado per...",37.787055,-122.408073,"[{'Personal': '8.2'}, {'Instalaciones y servic..."
3,26693,"Union Square, San Francisco, CA 94109, Estados...",Kasa La Monarca San Francisco está muy bien si...,"[Tenderloin Recreation Center\n500 m, WiFi gra...",37.788489,-122.412215,"[{'Personal': '7.6'}, {'Instalaciones y servic..."
4,26718,"777 North Vine Street, Hollywood, Los Ángeles,...",El Rodeway Inn near Melrose Ave se encuentra a...,"[Recepción 24 horas, Parking privado, WiFi gra...",34.085145,-118.326766,"[{'Personal': '7.7'}, {'Instalaciones y servic..."
...,...,...,...,...,...,...,...
551,36697,"2044 N Sycamore Ave , Hollywood, Los Ángeles, ...",Sycamore View Villa I se encuentra en el barri...,"[WiFi gratis, ¿Hay piscina?, ¿Hay spa?, ¿Se si...",34.107774,-118.342226,[]
552,36703,"3 Paseo Vespertino, Rancho Santa Margarita, CA...",King-30min to Airport-Self Check In-Parking-36...,"[¿Hay piscina?, WiFi gratis, ¿Hay spa?, ¿Se si...",33.640097,-117.597847,[]
553,36710,"25 Pacifica, Irvine, CA 92618, Estados Unidos",Irvine Spectrum/2 Bedrooms/2 Bathrooms/Apartme...,"[Piscina al aire libre, ¿Está abierta la pisci...",33.651016,-117.750711,"[{'Personal': '10'}, {'Instalaciones y servici..."
554,36713,"31861 Coast Highway, Laguna Beach, CA 92651, E...",OCEAN FRONT Beach HOUSE! Private Stairs to SAN...,"[WiFi gratis, ¿Hay piscina?, Habitaciones fami...",33.499589,-117.743265,[]


In [ ]:
attributes_3_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_3_3.csv", index_col=0)

In [8]:
attributes_1_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_1_3.csv", index_col=0)
attributes_2_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_2_3.csv", index_col=0)
attributes_3_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_3_3.csv", index_col=0)
california_hotels_attributes = pd.concat([attributes_1_3, attributes_2_3, attributes_3_3], ignore_index=True)
os.remove("../files/data/booking/attributes/california_hotels_attributes_1_3.csv")
os.remove("../files/data/booking/attributes/california_hotels_attributes_2_3.csv")
os.remove("../files/data/booking/attributes/california_hotels_attributes_3_3.csv")
california_hotels_attributes.to_csv("../files/data/booking/attributes/california_hotels_attributes.csv")

,Unnamed: 0,hotel_id,direction,description,attributes,latitude,longitude,scores
0,0,16,"24232 Tahoe Court, Laguna Niguel, CA 92677, Es...","Breathtaking views, close to the ocean and can...","['Habitaciones familiares', '¿Hay spa?', 'Park...",33.552129,-117.706753,[]
1,1,27,"220 20th Street, Newport Beach, CA 92663, Esta...","Surf Casita está en Newport Beach, a 3 min a p...","['Habitaciones familiares', 'Parking gratis', ...",33.609155,-117.926502,[]
2,2,35,"Pacific Beach, San Diego, 92109-5890, Estados ...",Upscale Beach Rowhome with Rooftop and Bay Vie...,"['Parking en el alojamiento', '¿Hay piscina?',...",32.790681,-117.240283,[]
3,3,39,"Julian Avenue 2054, San Diego, 92113, Estados ...",Victorian Retreat in Barrio Logan 1mi to Downt...,"['¿Hay spa?', '¿Hay piscina?', 'Parking gratis...",32.701546,-117.139944,[]
4,4,44,"4050 Albatross Street, Unit 4048, Hillcrest, S...",4048 Stylish condo in Hillcrest Mission Hills ...,"['Habitaciones familiares', '¿Hay spa?', '¿Hay...",32.746639,-117.166122,[]
...,...,...,...,...,...,...,...,...
1551,551,36697,"2044 N Sycamore Ave , Hollywood, Los Ángeles, ...",Sycamore View Villa I se encuentra en el barri...,"['WiFi gratis', '¿Hay piscina?', '¿Hay spa?', ...",34.107774,-118.342226,[]
1552,552,36703,"3 Paseo Vespertino, Rancho Santa Margarita, CA...",King-30min to Airport-Self Check In-Parking-36...,"['¿Hay piscina?', 'WiFi gratis', '¿Hay spa?', ...",33.640097,-117.597847,[]
1553,553,36710,"25 Pacifica, Irvine, CA 92618, Estados Unidos",Irvine Spectrum/2 Bedrooms/2 Bathrooms/Apartme...,"['Piscina al aire libre', '¿Está abierta la pi...",33.651016,-117.750711,"[{'Personal': '10'}, {'Instalaciones y servici..."
1554,554,36713,"31861 Coast Highway, Laguna Beach, CA 92651, E...",OCEAN FRONT Beach HOUSE! Private Stairs to SAN...,"['WiFi gratis', '¿Hay piscina?', 'Habitaciones...",33.499589,-117.743265,[]


Florida

In [52]:
florida_hotels = usa_hotels[usa_hotels["state"] == "Florida"]
florida_hotels_url_list = florida_hotels["reviews_url"].tolist()
florida_hotels_ids_list = florida_hotels["hotel_id"].tolist()

In [53]:
florida_hotels

,state,city,name,avg_score,price,reviews_url,hotel_id
9022,Florida,Tampa,Summertime Oasis - Tampa,100.0,272.596,https://www.booking.com/hotel/us/summertime-oa...,39
10037,Florida,Kissimmee,Thematic House w Pool in Resort w Clubhouse,99.0,314.104,https://www.booking.com/hotel/us/thematic-hous...,78
10035,Florida,Kissimmee,4BD Encanto home w/Free water park at Storey lake,99.0,361.086,https://www.booking.com/hotel/us/4bd-encanto-h...,79
9416,Florida,Orlando,Mejor ubicacion y confort Best location & comfort,99.0,257.354,https://www.booking.com/hotel/us/mejor-ubicaci...,221
9402,Florida,Orlando,Vista Cay Lakeview Elysium condo,99.0,617.551,https://www.booking.com/hotel/us/vista-cay-lak...,226
...,...,...,...,...,...,...,...
9395,Florida,Orlando,Luxury 5 Bedroom Villa with Secluded Pool/Spa ...,10.0,415.725,https://www.booking.com/hotel/us/luxury-5-bedr...,43229
9391,Florida,Orlando,"Delightful Home with Theme Room, Private Pool ...",10.0,360.977,https://www.booking.com/hotel/us/delightful-ho...,43230
9387,Florida,Orlando,Chic Orlando Family Getaway with Fenced Yard!,10.0,265.403,https://www.booking.com/hotel/us/chic-orlando-...,43235
9386,Florida,Orlando,Windsor Hills Resort- 204B,10.0,220.038,https://www.booking.com/hotel/us/windsor-hills...,43236


In [54]:
# Primera parte
florida_hotels_attributes_1_3 = scrapping.scrape_hotels_attributes(florida_hotels_url_list[:500], florida_hotels_ids_list[:500])
florida_hotels_attributes_1_3.to_csv("../files/data/booking/attributes/florida_hotels_attributes_1_3.csv")

1/500
2/500
3/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 79
4/500
5/500
Error obteniendo descripción para el hotel con ID 226: Message: stale element reference: stale element not found
  (Session info: headless chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C9F282B2+55298]
	(No symbol) [0x00007FF6C9E95E02]
	(No symbol) [0x00007FF6C9D505AB]
	(No symbol) [0x00007FF6C9D5509C]
	(No symbol) [0x00007FF6C9D56FDF]
	(No symbol) [0x00007FF6C9D570D0]
	(No symbol) [0x00007FF6C9D8CDFE]
	(No symbol) [0x00007FF6C9DB20AA]
	(No symbol) [0x00007FF6C9D880CF]
	(No symbol) [0x00007FF6C9DB22C0]
	(No symbol) [0x00007FF6C9DCAAA4]
	(No symbol) [0x00007FF6C9DB1E83]
	(No symbol) [0x00007FF6C9D8670A]
	(No symbol) [0x00007FF6C9D87964]
	GetHandleVerifier [0x00007FF6CA2A0AAB+3694587]
	GetHandleVerifier [0x00007

In [22]:
# Segunda parte
florida_hotels_attributes_2_3 = scrapping.scrape_hotels_attributes(florida_hotels_url_list[500:1000], florida_hotels_ids_list[500:1000])
florida_hotels_attributes_2_3.to_csv("../files/data/booking/attributes/florida_hotels_attributes_2_3.csv")

1/700
2/700
Error obteniendo atributos para el hotel con ID 27: Message: stale element reference: stale element not found
  (Session info: headless chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C9F282B2+55298]
	(No symbol) [0x00007FF6C9E95E02]
	(No symbol) [0x00007FF6C9D505AB]
	(No symbol) [0x00007FF6C9D5509C]
	(No symbol) [0x00007FF6C9D56FDF]
	(No symbol) [0x00007FF6C9D570D0]
	(No symbol) [0x00007FF6C9D8CDFE]
	(No symbol) [0x00007FF6C9DB20AA]
	(No symbol) [0x00007FF6C9D880CF]
	(No symbol) [0x00007FF6C9DB22C0]
	(No symbol) [0x00007FF6C9DCAAA4]
	(No symbol) [0x00007FF6C9DB1E83]
	(No symbol) [0x00007FF6C9D8670A]
	(No symbol) [0x00007FF6C9D87964]
	GetHandleVerifier [0x00007FF6CA2A0AAB+3694587]
	GetHandleVerifier [0x00007FF6CA2F728E+4048862]
	GetHandleVerifier [0x00007FF6CA2EF173+4015811]
	GetHandleVerifier [0x00007FF6C

,hotel_id,direction,description,attributes,latitude,longitude,scores
0,16,"24232 Tahoe Court, Laguna Niguel, CA 92677, Es...","El Breathtaking views, close to the ocean and ...","[Parking gratis, Habitaciones sin humo, Habita...",33.552129,-117.706753,[]
1,27,"220 20th Street, Newport Beach, CA 92663, Esta...","Set 8.1 km from Fashion Island, 11 km from Sou...",None,33.609155,-117.926502,[]
2,35,"Pacific Beach, San Diego, 92109-5890, Estados ...",El Upscale Beach Rowhome with Rooftop and Bay ...,"[Tostadora, Parking adaptado para personas de ...",32.790681,-117.240283,[]
3,39,"Julian Avenue 2054, San Diego, 92113, Estados ...",El Victorian Retreat in Barrio Logan 1mi to Do...,"[Parking gratis, Cocina, Utensilios de cocina,...",32.701546,-117.139944,[]
4,44,"4050 Albatross Street, Unit 4048, Hillcrest, S...",El 4048 Stylish condo in Hillcrest Mission Hil...,None,32.746639,-117.166122,[]
...,...,...,...,...,...,...,...
695,19850,"2576 Lombard Street, Marina District, San Fran...",Este motel está situado junto a la famosa call...,"[Tetera/cafetera en todas las habitaciones, Pa...",37.798998,-122.444005,None
696,19856,"3115 Camino Del Rio Court, Bakersfield, CA 933...","Con fácil acceso a la autopista 99, el hotel C...",None,35.381132,-119.047666,[{'Suite con cama extragrande - No fumadores':...
697,19859,"19480 Quinn Road, Bakersfield, CA 93308, Estad...",El Holiday Inn Express & Suites Bakersfield Ai...,"[Tetera/cafetera en todas las habitaciones, Gi...",35.442841,-119.082435,None
698,19868,"2430 Newport Boulevard, Costa Mesa, CA 92627, ...","Este hotel de Costa Mesa, California, está sit...",None,33.657790,-117.901813,None


In [23]:
# Tercera parte
florida_hotels_attributes_3_3 = scrapping.scrape_hotels_attributes(florida_hotels_url_list[1000:], florida_hotels_ids_list[1000:])
florida_hotels_attributes_3_3.to_csv("../files/data/booking/attributes/florida_hotels_attributes_3_3.csv")

1/856
Error obteniendo atributos para el hotel con ID 19872: Message: stale element reference: stale element not found
  (Session info: headless chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C9F282B2+55298]
	(No symbol) [0x00007FF6C9E95E02]
	(No symbol) [0x00007FF6C9D505AB]
	(No symbol) [0x00007FF6C9D5509C]
	(No symbol) [0x00007FF6C9D56FDF]
	(No symbol) [0x00007FF6C9D570D0]
	(No symbol) [0x00007FF6C9D8CDFE]
	(No symbol) [0x00007FF6C9DB20AA]
	(No symbol) [0x00007FF6C9D880CF]
	(No symbol) [0x00007FF6C9DB22C0]
	(No symbol) [0x00007FF6C9DCAAA4]
	(No symbol) [0x00007FF6C9DB1E83]
	(No symbol) [0x00007FF6C9D8670A]
	(No symbol) [0x00007FF6C9D87964]
	GetHandleVerifier [0x00007FF6CA2A0AAB+3694587]
	GetHandleVerifier [0x00007FF6CA2F728E+4048862]
	GetHandleVerifier [0x00007FF6CA2EF173+4015811]
	GetHandleVerifier [0x00007FF6C9FC

KeyboardInterrupt: 

In [ ]:
# Concatenar los resultados
florida_hotels_attributes = pd.concat([florida_hotels_attributes_1_2, florida_hotels_attributes_2_2, florida_hotels_attributes_3_3], ignore_index=True)

# Guardar el archivo final
output_path_combined = "../files/data/booking/attributes/florida_hotels_attributes.csv"
florida_hotels_attributes.to_csv(output_path_combined, index=False)

# Eliminar archivos temporales
os.remove("../files/data/booking/attributes/florida_hotels_attributes_1_2.csv")
os.remove("../files/data/booking/attributes/florida_hotels_attributes_2_2.csv")